In [1]:
import gc
import os

import keras
import keras_models
import numpy as np
import pandas as pd
import utils
from gensim.models import KeyedVectors
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD, Adadelta, Adam, Nadam, RMSprop
from keras.preprocessing import sequence, text
from nltk.corpus import stopwords
from tqdm import tqdm

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45

/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is

In [2]:
n_folds = 5
n_bags = 1
split_size = 0.1
max_features = 300000
nb_words = max_features
sequence_length = 196
embedding_dim = 300
lstm_units = 256
bidirectional = True

run_prefix = 'Stacker1'
src = '../data/'
model_name = 'MLPbasic'
optimizer = 'Nadam'
data_type = 'BasicClean'
kfold_run = 1
batch_size = 256
importance = 0
stratify = 0
save_models = 0
load_models = 0
save_oof = 0
prepare_submission = 0

src_oof = 'oof/'
src_features = '../data/features/'
run_kfold = True
split_for_validation = True
add_features = False

In [3]:
if bidirectional and 'LSTM' in model_name or bidirectional and 'GRU' in model_name:
    run_prefix = 'Bidirectional{}'.format(run_prefix)
if kfold_run:
    general_run_name = '{}{}fold_BS{}_{}'.format(
        run_prefix, n_folds, batch_size, optimizer)
else:
    general_run_name = '{}{}bag_BS{}_{}'.format(
        run_prefix, n_bags, batch_size, optimizer)


if len(data_type) > 0:
    general_run_name += '_{}'.format(data_type)
if importance:
    general_run_name += '_ImportanceTrain'
if stratify and kfold_run:
    general_run_name += '_Stratified'

run_name = '{}{}'.format(model_name, general_run_name)
print('Run name: {}'.format(run_name))


model_callbacks = [EarlyStopping(monitor='val_loss', patience=10, verbose=1),
                   ReduceLROnPlateau(monitor='val_loss', factor=0.5, verbose=1,
                                     patience=6, min_lr=1e-5)]

if optimizer == 'Adam':
    optimizer = Adam(lr=1e-4, decay=1e-3)
    # optimizer = 'adam'
if optimizer == 'Nadam':
    optimizer = Nadam(lr=1e-4, schedule_decay=1e-3)
    # optimizer = 'nadam'
if optimizer == 'SGD':
    optimizer = SGD(lr=1e-3, momentum=0.9,
                    decay=1e-4, nesterov=True)

Run name: MLPbasicStacker15fold_BS256_Nadam_BasicClean


In [5]:
if run_kfold:
    prefix = 'KFold'
else:
    prefix = 'Bag'


train = pd.read_pickle("../data/train_basic_clean.pkl")
test = pd.read_pickle("../data/test_basic_clean.pkl")
target_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

oof_tr, oof_te = utils.load_predictions(src_oof, contains='Check')
oof_tr, oof_te = np.asarray(oof_tr), np.asarray(oof_te)
print(oof_tr.shape, oof_te.shape)

X_train_oof = pd.DataFrame()
X_test_oof = pd.DataFrame()

for i in range(oof_tr.shape[0]):
    oof_tr_part = oof_tr[i][:, :, 0]
    oof_te_part = oof_te[i].mean(axis=-1)[:, :]
    X_train_oof = pd.concat([X_train_oof, pd.DataFrame(oof_tr_part)], axis=1)
    X_test_oof = pd.concat([X_test_oof, pd.DataFrame(oof_te_part)], axis=1)


f_cols = []

for i in range((X_train_oof.shape[1] // 6)):
    f_cols.extend(['{}_preds_model{}'.format(target_columns[x], i) for x in range(len(target_columns))])

X_train_oof.columns = f_cols
X_test_oof.columns = f_cols


if add_features:
    data_tokenized = pd.read_pickle(src_features + 'data_TokenizedSentences160.pkl')
    data_badwords300 = pd.read_pickle(src_features + 'data_Binary300Badwords.pkl')
    data_textStatistics = pd.read_pickle(src_features + 'data_TextStatistics.pkl')
    data_transformations = pd.read_pickle(src_features + 'data_20dim_SVDLSA.pkl')

    data_tokenized.reset_index(drop=True, inplace=True)
    
    X = pd.concat([data_tokenized, data_badwords300, data_textStatistics, data_transformations], axis=1)
    X['badwordsCount'] = data_badwords300.sum(axis=1)

    X_train = X.iloc[:train.shape[0], :]
    X_test = X.iloc[train.shape[0]:, :]
    y_train = train[target_columns].values

    features = np.setdiff1d(X_train.columns, target_columns)

    X_train = pd.concat([X_train[features], X_train_oof], axis=1)
    X_test = pd.concat([X_test[features], X_test_oof], axis=1)
    
    del X, test
    del data_tokenized, data_badwords300
    del data_textStatistics, data_transformations
    gc.collect()

else:
    X_train = X_train_oof
    y_train = train[target_columns].values
    X_test = X_test_oof
    

print(X_train.shape, X_test.shape)


Train files:

['oof/train/GRUattentionBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_fromCheck_121.12740.pkl',
 'oof/train/GRUbasicBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_fromCheck_110.98027.pkl',
 'oof/train/GRUconcatBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_fromCheck_94.57426.pkl',
 'oof/train/LSTMattentionBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Adam_fromCheck_116.43669.pkl',
 'oof/train/LSTMconcatBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Adam_fromCheck_113.36922.pkl']

Test files:

['oof/test/GRUattentionBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_fromCheck_121.12740.pkl',
 'oof/test/GRUbasicBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_fromCheck_110

In [6]:
model_parameters = {
    'lstm_units': lstm_units,
    'dropout_rate': 0.2,
    'bidirectional': bidirectional,
    'nb_words': nb_words,
    'embedding_dim': embedding_dim,
    'num_columns': X_train.shape[1],
    # 'embedding_matrix': embedding_matrix,
    'sequence_length': sequence_length,
    'optimizer': optimizer,
    'loss': utils.tf_roc_auc,
}

pipeline_parameters = {
    'model_name': getattr(keras_models, model_name),
    'predict_test': True,
    'number_epochs': 100,
    'batch_size': batch_size,
    'seed': 1337,
    'shuffle': True,
    'verbose': True,
    'run_save_name': run_name,
    'load_keras_model': load_models,
    'save_model': save_models,
    'save_history': True,
    'save_statistics': True,
    'output_statistics': True,
    'src_dir': os.getcwd(),
}

if kfold_run:
    oof_train, oof_test = utils.run_parametrized_kfold(X_train.values, y_train, 
                                                       X_test.values,
                                                       pipeline_parameters,
                                                       model_parameters,
                                                       model_callbacks=model_callbacks,
                                                       n_folds=n_folds,
                                                       importance_training=importance,
                                                       save_oof=save_oof)
    print(oof_train.shape, oof_test.shape)
else:
    oof_valid, oof_test = utils.run_parametrized_bagging(X_train, y_train,
                                                         X_test=X_test,
                                                         pipeline_parameters=pipeline_parameters,
                                                         model_parameters=model_parameters,
                                                         model_callbacks=model_callbacks,
                                                         n_bags=n_bags,
                                                         split_size=split_size,
                                                         importance_training=importance)
    print(oof_valid.shape, oof_test.shape)

Running parametrized KFold run.
Running: MLPbasicStacker15fold_BS256_Nadam_BasicClean
OOF train predictions shape: (159571, 6, 1)
OOF test predictions shape: (153164, 6, 5)
Training on fold: 1 



/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.sign

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30)                120       
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               7936      
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 256)               256       
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
__________

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.sign

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.sign

...

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.sign

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 30)                0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 30)                120       
_________________________________________________________________
dropout_13 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               7936      
_________________________________________________________________
p_re_lu_13 (PReLU)           (None, 256)               256       
_________________________________________________________________
batch_normalization_14 (Batc (None, 256)               1024      
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
__________

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.sign

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.sign

Train on 127657 samples, validate on 31914 samples
Epoch 1/100
127657/127657 [==============================] - 5s 41us/step - loss: 279.5383 - acc: 0.3194 - val_loss: 69.8343 - val_acc: 0.3382

 ROC-AUC - epoch: 1 - score: 0.979213 

Epoch 2/100
127657/127657 [==============================] - 4s 31us/step - loss: 109.4376 - acc: 0.4064 - val_loss: 62.7537 - val_acc: 0.4007

 ROC-AUC - epoch: 2 - score: 0.980595 

Epoch 3/100
127657/127657 [==============================] - 4s 31us/step - loss: 94.9667 - acc: 0.4597 - val_loss: 58.8572 - val_acc: 0.4386

 ROC-AUC - epoch: 3 - score: 0.981226 

Epoch 4/100
127657/127657 [==============================] - 4s 30us/step - loss: 88.0119 - acc: 0.4937 - val_loss: 55.9335 - val_acc: 0.4750

 ROC-AUC - epoch: 4 - score: 0.981570 

Epoch 5/100
127657/127657 [==============================] - 4s 32us/step - loss: 84.7417 - acc: 0.5275 - val_loss: 52.9949 - val_acc: 0.4951

 ROC-AUC - epoch: 5 - score: 0.982120 

Epoch 6/100
127657/127657 [=====

In [13]:
utils.output_submission(
        oof_test.mean(axis=-1), run_name, save=True)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.994582,0.838217,0.968852,0.772463,0.969362,0.866074
1,0000247867823ef7,0.663043,0.144427,0.372096,0.210325,0.432950,0.254375
2,00013b17ad220c46,0.695902,0.348391,0.588249,0.463334,0.598019,0.561232
3,00017563c3f7919a,0.377982,0.077353,0.266755,0.063698,0.311970,0.095862
4,00017695ad8997eb,0.731578,0.332978,0.597268,0.559649,0.624729,0.579527
5,0001ea8717f6de06,0.349854,0.083074,0.216944,0.065285,0.270267,0.069183
6,00024115d4cbde0f,0.777514,0.304776,0.596479,0.503285,0.664335,0.584461
7,000247e83dcc1211,0.904294,0.504403,0.787111,0.596406,0.811055,0.681050
8,00025358d4737918,0.813008,0.380032,0.710142,0.544012,0.726670,0.616330
9,00026d1092fe71cc,0.691031,0.175417,0.509381,0.261079,0.585193,0.364609
